In [1]:
''' 
  This is to create sym links of file needed in the notebook
  used for devlopment only!!!
'''
# !ln -s ./drive/My\ Drive/HornMorph/
# !ln -s ./drive/My\ Drive/AMh2ESLTest/raw_data/
# !ln -s ./drive/My\ Drive/AMh2ESLTest/processed_data/
# !ln -s ./drive/My\ Drive/AMh2ESLTest/embedding_checkpoints/
# !ln -s ./drive/My\ Drive/AMh2ESLTest/translator_helper.py
# !ln -s ./drive/My\ Drive/AMh2ESLTest/preprocess_util.py
# # !ln -s ./drive/My\ Drive/AMh2ESLTest/word2vec_helper.py
# !ln -s ./drive/My\ Drive/AMh2ESLTest/amh_vocab_mapping.p
# !ln -s ./drive/My\ Drive/AMh2ESLTest/preprocssed_data.p

' \n  This is to create sym links of file needed in the notebook\n  used for devlopment only!!!\n'

In [2]:
import translator_helper as helper
import os
import preprocess_util as putil
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from tensorflow import keras
print(tf.__version__)
if not tf.test.gpu_device_name():
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

2.2.0
Default GPU Device: /device:GPU:0


In [3]:
source_path = "./raw_data/parallel_sentence_corpus/amh2.txt"
target_path = "./raw_data/parallel_sentence_corpus/amh2.txt"
source_vocab_mapping = "./amh_vocab_mapping.p"
target_vocab_mapping = "./amh_vocab_mapping.p"
PREPROCESS_SAVE_PATH = "preprocssed_data.p"

In [4]:
#loading src language vlocabulary and target language vocabulay
src_int_to_vocab, src_vocab_to_int = putil.load_file(source_vocab_mapping)
tgt_int_to_vocab, tgt_vocab_to_int = putil.load_file(target_vocab_mapping)

In [5]:
helper.preprocess_and_save(source_path, target_path,
                           src_int_to_vocab, src_vocab_to_int,
                           tgt_int_to_vocab, tgt_vocab_to_int,
                           PREPROCESS_SAVE_PATH)
print("processed data saved...")
# data is cleaned, indexed and processed!
# chekpoint saved

processed data saved...


In [6]:
# loading all processed data
# src_int_text: src language sentences
(src_int_text, tgt_int_text), (src_vocab_to_int, tgt_vocab_to_int), (src_int_to_vocab, tgt_int_to_vocab)  = helper.load_preprocessed_data(PREPROCESS_SAVE_PATH)

In [7]:
def load_dataset(src_int_sent, tgt_int_sent, src_pad_id, tgt_pad_id):
    max_src_lang = max([len(sen) for sen in src_int_sent])
    max_tgt_lang = max([len(sen) for sen in tgt_int_sent]) 
    #padding every sentence with max length of the longest sentence in the batch
    padded_src_sent = tf.keras.preprocessing.sequence.pad_sequences(src_int_sent, 
                                                                 maxlen=max_src_lang,
                                                                 padding='post',
                                                                 value= src_pad_id)
    padded_tgt_sent = tf.keras.preprocessing.sequence.pad_sequences(tgt_int_sent, 
                                                                  maxlen=max_tgt_lang, 
                                                                  padding='post',
                                                                  value=tgt_pad_id)
    return padded_src_sent, padded_tgt_sent, max_src_lang, max_tgt_lang

In [8]:
def get_embedding_mat(embedding_size, vocab_size, load_path):
  embed_graph = tf.Graph()
  with embed_graph.as_default():
    inputs = tf.compat.v1.placeholder(tf.int32, [None], name='inputs')
    labels = tf.compat.v1.placeholder(tf.int32, [None, None], name='labels')
    embedding = tf.Variable(tf.compat.v1.truncated_normal([vocab_size, embedding_size]))
  with embed_graph.as_default():
    saver = tf.compat.v1.train.Saver()

  with tf.compat.v1.Session(graph=embed_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint(load_path))
    embed_mat = sess.run(embedding)
  #tf.reset_default_graph()
  return embed_mat

In [9]:
def gru(units):
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [10]:
class Encoder(tf.keras.Model):
    def __init__(self, embed_mat, rnn_size, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.rnn_size = rnn_size
        self.embed_mat = embed_mat
        self.gru = gru(self.rnn_size)
        
    def call(self, enc_input, hidden):
        embeded_input = tf.nn.embedding_lookup(self.embed_mat, enc_input)
        output, state = self.gru(embeded_input, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.rnn_size))

In [11]:
class Decoder(tf.keras.Model):
    def __init__(self, tgt_vocab_size, embed_mat, rnn_size, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.rnn_size = rnn_size
        self.embed_mat = embed_mat
        self.gru = gru(self.rnn_size)

        self.output_layer = tf.keras.layers.Dense(tgt_vocab_size)
        # used for attention 
        self.W1 = tf.keras.layers.Dense(self.rnn_size)
        self.W2 = tf.keras.layers.Dense(self.rnn_size)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_input, hidden, enc_output):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        dec_embeded_input = tf.nn.embedding_lookup(self.embed_mat, dec_input)
        x = tf.concat([tf.expand_dims(context_vector, 1), dec_embeded_input], axis=-1)
        
        output, state = self.gru(x)        
        output = tf.reshape(output, (-1, output.shape[2]))
        
        x = self.output_layer(output)
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [12]:
# preparing the dataset and setting hyper parameters
input, target, max_src_len, max_tgt_len = load_dataset(src_int_text, tgt_int_text, src_vocab_to_int['<PAD>'], tgt_vocab_to_int['<PAD>'])
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input, target, test_size=0.1, random_state = 101)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 256
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
rnn_size = 256
lr = 0.001

src_vocab_size = len(src_vocab_to_int)
vocab_tar_size = len(tgt_vocab_to_int)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)



In [13]:
# loading the embedding matrix
src_embed_mat = get_embedding_mat(256, len(src_int_to_vocab),"embedding_checkpoints")
tgt_embed_mat = get_embedding_mat(256, len(src_int_to_vocab),"embedding_checkpoints")

# intitalizing the encoder and decoder
encoder = Encoder(src_embed_mat, 256, 256)
decoder = Decoder(len(tgt_vocab_to_int), tgt_embed_mat, 256, 256)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Restoring parameters from embedding_checkpoints/amh_emebdding.ckpt
INFO:tensorflow:Restoring parameters from embedding_checkpoints/amh_emebdding.ckpt
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [14]:
optimizer = tf.train.AdamOptimizer()
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)
 

In [15]:
checkpoint_dir = './training_checkpoints3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder) 

In [ ]:
import time
EPOCHS = 100
for epoch in range(EPOCHS):
    start = time.time() 
    hidden = encoder.initialize_hidden_state()
    total_loss = 0 
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0      
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)            
            dec_hidden = enc_hidden            
            dec_input = tf.expand_dims([tgt_vocab_to_int['<GO>']] * BATCH_SIZE, 1)       

            for t in range(0, targ.shape[1]):

                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                loss += loss_function(targ[:, t], predictions)
                
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        total_loss += batch_loss 
        variables = encoder.variables + decoder.variables  
        gradients = tape.gradient(loss, variables) 
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    checkpoint.save(file_prefix = checkpoint_prefix) 
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    

In [ ]:
def evaluate(inputs, encoder, decoder, max_length_tgt_to_trans): 
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    init_hidden = [tf.zeros((1, 256))]
    enc_out, enc_hidden = encoder(inputs, init_hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tgt_vocab_to_int['<GO>']], 0)

    for t in range(max_length_tgt_to_trans): 
        translation, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_out)  
        translation = tf.argmax(translation[0]).numpy()
        result += tgt_int_to_vocab[translation] + ' '
        if tgt_int_to_vocab[translation] == '<EOS>':
            return result
        dec_input = tf.expand_dims([translation], 0)

    return result